In [25]:
import io
from urllib.request import urlopen, Request
import sqlite3

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.io.img_tiles as cimgt
import pandas as pd 

%matplotlib inline


In [26]:
# Load from SQL database
conn = sqlite3.connect("observation_repo.db")
query = "SELECT * FROM weather_data"
df = pd.read_sql_query(query, conn)
display(df) 

,id,date_time,stid,elevation,air_temp,dew_point_temperature,pressure,wind_direction,wind_speed,wind_u,wind_v,snow_depth,solar_radiation,ozone_concentration,air_density
0,1,2022-09-29T22:15:00+00:00,KVEL,1599.648,24.0,6.99,84029.07,190.0,2.572,0.447,2.533,NaN,NaN,NaN,0.985092
1,2,2022-09-29T22:20:00+00:00,KVEL,1599.648,24.0,6.99,84029.07,170.0,2.058,-0.357,2.027,NaN,NaN,NaN,0.985092
2,3,2022-09-29T22:25:00+00:00,KVEL,1599.648,24.0,6.99,84029.07,190.0,3.087,0.536,3.040,NaN,NaN,NaN,0.985092
3,4,2022-09-29T22:30:00+00:00,KVEL,1599.648,24.0,5.99,84029.07,200.0,2.058,0.704,1.934,NaN,NaN,NaN,0.985095
4,5,2022-09-29T22:35:00+00:00,KVEL,1599.648,24.0,5.99,84029.07,200.0,1.543,0.528,1.450,NaN,NaN,NaN,0.985095
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7835255,7835256,2023-09-24T22:35:00+00:00,K40U,1877.200,21.8,-3.90,81487.07,110.0,1.543,-1.450,0.528,NaN,NaN,NaN,0.962440
7835256,7835257,2023-09-24T22:55:00+00:00,K40U,1877.200,21.8,-4.30,81487.07,110.0,2.572,-2.417,0.880,NaN,NaN,NaN,0.962440
7835257,7835258,2023-09-24T23:15:00+00:00,K40U,1877.200,21.7,-4.40,81487.07,140.0,2.058,-1.323,1.577,NaN,NaN,NaN,0.962767
7835258,7835259,2023-09-24T23:35:00+00:00,K40U,1877.200,21.7,-4.00,81487.07,100.0,2.058,-2.027,0.357,NaN,NaN,NaN,0.962766


In [27]:
def image_spoof(self, tile):
    """Thanks to Joshua Hrisko 
    https://makersportal.com/blog/2020/4/24/geographic-visualizations-in-python-with-cartopy
    """    
    # get the url of the street map API
    url = self._image_url(tile) 
    req = Request(url) 
    req.add_header('User-agent','Anaconda 3') 
    fh = urlopen(req)
    im_data = io.BytesIO(fh.read()) 
    fh.close() 
    # open image with PIL, set format, reformat for Cartopy
    img = Image.open(im_data) 
    img = img.convert(self.desired_tile_form)
    return (img, self.tileextent(tile), 'lower') 

In [28]:
cimgt.Stamen.get_image = image_spoof  # reformat web request for street map spoofing
osm_img = cimgt.Stamen('terrain')  # spoofed, downloaded street map

In [29]:
# Find those stations with snow depth, and plot their locations
df_snow = df[~df['snow_depth'].isna()]
display(df_snow)

,id,date_time,stid,elevation,air_temp,dew_point_temperature,pressure,wind_direction,wind_speed,wind_u,wind_v,snow_depth,solar_radiation,ozone_concentration,air_density
113832,113833,2022-09-29T23:00:00+00:00,CWHU1,3191.392,8.700,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
113833,113834,2022-09-30T00:00:00+00:00,CWHU1,3191.392,6.400,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
113835,113836,2022-09-30T02:00:00+00:00,CWHU1,3191.392,1.900,NaN,NaN,NaN,NaN,NaN,NaN,4775.2,NaN,NaN,NaN
113836,113837,2022-09-30T03:00:00+00:00,CWHU1,3191.392,2.500,NaN,NaN,NaN,NaN,NaN,NaN,4775.2,NaN,NaN,NaN
113837,113838,2022-09-30T04:00:00+00:00,CWHU1,3191.392,3.700,NaN,NaN,NaN,NaN,NaN,NaN,4775.2,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7705238,7705239,2023-09-24T23:20:00+00:00,UT43W,2198.224,18.433,-1.14,NaN,109.0,1.924,-1.819,0.626,0.0,NaN,NaN,NaN
7705239,7705240,2023-09-24T23:30:00+00:00,UT43W,2198.224,18.250,-2.22,NaN,118.0,2.459,-2.171,1.154,0.0,NaN,NaN,NaN
7705240,7705241,2023-09-24T23:40:00+00:00,UT43W,2198.224,17.944,1.39,NaN,133.6,1.790,-1.296,1.234,0.0,NaN,NaN,NaN
7705241,7705242,2023-09-24T23:50:00+00:00,UT43W,2198.224,17.872,-0.60,NaN,134.8,1.996,-1.416,1.406,0.0,NaN,NaN,NaN


In [30]:
df_snow[df_snow["latitude"]][0]

KeyError: 'latitude'

In [ ]:
unique_stid = df_snow["stid"].unique()
# Create lookup of station to lat/lon
latlonlookup = {}

for stid in unique_stid:
    # Take first row with this stid and assign that latlon
    latlonlookup[stid] = df_snow[df_snow["latitude"]][0]
# Look up that station
print(unique_stid, df_snow)

def plot_stid_locs(stids, lats, lons):
    
    fig = plt.figure(figsize=(12, 9))  # open matplotlib figure
    ax = plt.axes(projection=osm_img.crs)  # project using coordinate reference system (CRS) of street map
    
    _lats = []
    _lons = []
    for s in stids:
        lon = lons[s]
        lat = lats[s]
        elev = elevs[s]
    
        ax.text(lon, lat, s, transform=ccrs.PlateCarree(),)# **text_kwargs)
        _lats.append(lat)
        _lons.append(lon)
    sc = ax.scatter(lons,lats,transform=ccrs.PlateCarree(),c=elevs)
    cbar = fig.colorbar(sc)
    
    ax.add_feature(cfeature.STATES.with_scale("10m"))
    ax.add_feature(cfeature.RIVERS.with_scale("10m"))
    # ax.stock_img()
    # ax.add_feature(cfeature.NaturalEarthFeature("physical","",scale="10m"))
    
    extent = [-110.6,-108.7,39.65,41.05]
    ax.set_extent(extent)  # set extents
    
    # empirical solve for scale based on zoom
    scale = np.ceil(-np.sqrt(2) * np.log(np.divide((extent[1] - extent[0]) / 2.0, 350.0)))
    scale = (scale < 20) and scale or 19  # scale cannot be larger than 19
    ax.add_image(osm_img, int(scale))  # add OSM with zoom specification
    
    ax.set_title("Station Locations", loc="left", fontweight="bold")
    fig.tight_layout()
    # fig.show()
    
plot_stid_locs(unique_stid)